In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

import time
import re
import json
import requests
import urllib
from bs4 import BeautifulSoup
import urllib.request

with open('config.json') as config_file:
    data = json.load(config_file)

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
email = data['email']
password = data['password']
keywords = data['keywords']
location = data['location']
driver = webdriver.Chrome(options=options, executable_path=data['driver_path'])


In [8]:
# go to the LinkedIn login url
driver.get("https://www.linkedin.com/login")

# introduce email and password and hit enter
login_email = driver.find_element_by_name('session_key')
login_email.clear()
login_email.send_keys(email)
login_pass = driver.find_element_by_name('session_password')
login_pass.clear()
login_pass.send_keys(password)
login_pass.send_keys(Keys.RETURN)
time.sleep(1)
speicher = driver.find_element_by_xpath('//*[@id="remember-me-prompt__form-primary"]/button')
speicher.click()
time.sleep(5)

In [9]:
jobs_link = driver.find_element_by_link_text('Jobs')
jobs_link.click()
time.sleep(5)
# search based on keywords and location and hit enter
search_keywords = driver.find_element_by_xpath('//*[@class="jobs-search-box__text-input jobs-search-box__keyboard-text-input"]')

#search_keywords.clear()
search_keywords.send_keys(keywords)

search_location = driver.find_element_by_xpath('//*[@class="jobs-search-box__text-input"]')
#search_location.clear()
search_location.send_keys(location)
search_location.send_keys(Keys.RETURN)
time.sleep(2)
easy_apply_button = driver.find_element_by_xpath("//button[@aria-label='Easy Apply filter.']")
easy_apply_button.click()

In [10]:
total_results = driver.find_element_by_class_name("display-flex.t-12.t-black--light.t-normal")
total_results_int = int(total_results.text.split(' ',1)[0].replace(",",""))
print(total_results_int)

154


In [11]:
url = driver.current_url
results = driver.find_elements_by_class_name("jobs-search-results__list-item.occludable-update.p0.relative.ember-view")


In [12]:
def submit_apply(job_add):
    """This function submits the application for the job add found"""
    print('You are applying to the position of: ', job_add.text)
    job_add.click()
    # click on the easy apply button, skip if already applied to the position
    try:
        in_apply = driver.find_element_by_xpath("//button[@data-control-name='jobdetails_topcard_inapply']")
        in_apply.click()
    except NoSuchElementException:
        print('You already applied to this job, go to next...')
        pass
    driver.implicitly_wait(1)
    # try to submit if submit application is available...
    try:
        try:
            continue_job = driver.find_element_by_xpath("//button[@aria-label='Continue to next step']")
            continue_job.click()
            driver.implicitly_wait(1)
        except:
            print('direct apply..')
            pass
        try:
            continue_job = driver.find_element_by_xpath("//button[@aria-label='Continue to next step']")
            continue_job.click()
            driver.implicitly_wait(1)
        except:
            print('direct apply..')
            pass
        try:
            select = Select(driver.find_element_by_id('urn:li:fs_easyApplyFormElement:(urn:li:fs_normalized_jobPosting:2462810884,21613107,multipleChoice)'))
            # select by visible text
            select.select_by_visible_text('Verhandlungssicher')
            driver.implicitly_wait(1)
        except:
            print('filling options..')
            pass
        try:
            input_field = driver.find_elements_by_class_name("ember-text-field.ember-view.fb-single-line-text__input")
            for f in input_field:
                f.clear()
                f.send_keys('2')
            driver.implicitly_wait(1)
        except:
            print('sending inputs..')
            pass
        try:
            continue_job = driver.find_element_by_xpath("//button[@aria-label='Review your application']")
            continue_job.click()
            driver.implicitly_wait(1)
        except:
            print('reviewing..')
            pass
        finally:
            submit = driver.find_element_by_xpath("//button[@data-control-name='submit_unify']")
            submit.send_keys(Keys.RETURN)
            driver.implicitly_wait(1)
            try:
                closing = driver.find_element_by_xpath("//button[@aria-label='Dismiss']")
                closing.click()
                driver.implicitly_wait(1)
            except:
                print('no close popup')
                pass  
    # ... if not available, discard application and go to next
    except NoSuchElementException:
        print('Not direct application, going to next...')
        try:
            discard = driver.find_element_by_xpath("//button[@data-test-modal-close-btn]")
            discard.send_keys(Keys.RETURN)
            #driver.implicitly_wait(1)
            discard_confirm = driver.find_element_by_xpath("//button[@data-test-dialog-primary-btn]")
            discard_confirm.send_keys(Keys.RETURN)
            #driver.implicitly_wait(1)
        except NoSuchElementException:
            applied = True
    driver.implicitly_wait(1)
while True:
    for result in results:
        try:
            hover = ActionChains(driver).move_to_element(result)
            hover.perform()
            driver.implicitly_wait(1)
            titles = result.find_elements_by_class_name('full-width.artdeco-entity-lockup__title.ember-view')
        except:
            pass
        for title in titles:
            try:
                submit_apply(title)
                if applied:
                    continue
                driver.implicitly_wait(1)
            except:
                pass
    # if there is more than one page, find the pages and apply to the results of each page
    if total_results_int > 24:
            driver.implicitly_wait(1)
            # find the last page and construct url of each page based on the total amount of pages
            find_pages = driver.find_elements_by_class_name("artdeco-pagination__indicator.artdeco-pagination__indicator--number")
            print('scanning..')
            total_pages = find_pages[len(find_pages)-1].text
            total_pages_int = int(re.sub(r"[^\d.]", "", total_pages))
            get_last_page = driver.find_element_by_xpath("//button[@aria-label='Page "+str(total_pages_int)+"']")
            get_last_page.send_keys(Keys.RETURN)
            driver.implicitly_wait(1)
            last_page = driver.current_url
            total_jobs = int(last_page.split('start=',1)[1])
            # go through all available pages and job offers and apply
            for page_number in range(25,total_jobs+25,25):
                driver.get(current_page+'&start='+str(page_number))
                driver.implicitly_wait(1)
                results_ext = driver.find_elements_by_class_name("occludable-update.artdeco-list__item--offset-4.artdeco-list__item.p0.ember-view")
                for result_ext in results_ext:
                    hover_ext = ActionChains(driver).move_to_element(result_ext)
                    hover_ext.perform()
                    titles_ext = result_ext.find_elements_by_class_name('job-card-search__title.artdeco-entity-lockup__title.ember-view')
                    for title_ext in titles_ext:
                        submit_apply(title_ext)

You are applying to the position of:  IT-Systemadministrator (m/w/d)
You already applied to this job, go to next...
direct apply..
direct apply..
filling options..
reviewing..
Not direct application, going to next...
You are applying to the position of:  HR Controller & Analyst (w/m/d)
direct apply..
direct apply..
filling options..
reviewing..
Not direct application, going to next...
You are applying to the position of:  Softwareentwickler/in für Automatisierungstechnik (m/w/d)
You already applied to this job, go to next...
direct apply..
direct apply..
filling options..
reviewing..
Not direct application, going to next...
You are applying to the position of:  Trainee (m/w/d) Atlassian Consultant
direct apply..
direct apply..
filling options..
reviewing..
Not direct application, going to next...
You are applying to the position of:  Platform Operations Engineer
filling options..
reviewing..
Not direct application, going to next...
You are applying to the position of:  Manager für Vert

IndexError: list index out of range